# Imports and packages

In [2]:
## helpful packages

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python3 -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## suppresses the traceback of an error
%xmode Minimal

Exception reporting mode: Minimal


# Text analysis of Department of Justice (DOJ) press releases (50 points total)

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases
    
- See here for a codebook: https://docs.google.com/spreadsheets/d/1UopmSvFGrwJvz_c3Plh32Yxkqwff64oS_CcpfATOV8k/edit?usp=sharing

In [3]:
## first, unzip the combined.json.zip file
## then, run this code to load the unzipped json file and convert to a dataframe
## and convert some of the attributes from lists to values
## make sure to change the pathname if you need to
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (16 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [4]:
# subset to one press release 
subset = doj.contents.loc[doj.id == "17-1204"]

# convert to a string
pharma = subset.to_string()

# check this worked
type(pharma)
#pharma

str

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

In [5]:
## A

# remove all punctuation / digits (so can use .isalpha() to subset)

# tokenize the string
pharm_tokens = word_tokenize(pharma).copy()

# check this worked
#pharm_tokens
#type(pharm_tokens)

# use list iteration to subset to only alphabetical tokens
processed = [token for token in pharm_tokens if token.isalpha()]

In [6]:
## B

# speech tagging
tagged = pos_tag(processed)

# check this worked
tagged[0:4]

[('The', 'DT'), ('founder', 'NN'), ('and', 'CC'), ('majority', 'NN')]

In [7]:
## C

# Using the output from B, extract the adjectives and sort those adjectives from most occurrences to
# fewest occurrences. 
adj = ["JJ", "JJR", "JJS"]
adj_list = [one_tok[0] for one_tok in tagged if one_tok[1] in adj]
pharm_adj = pd.DataFrame(adj_list, columns = ['adj']).groupby("adj").size().reset_index()
pharm_adj.columns = ['adj', "count"] 
pharm_adj_sorted = pharm_adj.sort_values(by = "count", ascending = False).copy()

#Print a dataframe with the 5 most frequent adjectives and their counts in the pharma release. 
pharm_adj_sorted.head(5)

,adj,count
19,former,8
40,opioid,5
38,nationwide,4
41,other,3
4,addictive,3


## 1.2 named entity recognition (3 points)



A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`. Here's some background on what RICO means: https://www.justia.com/criminal/docs/rico/ 

In [8]:
## A

# extract the named entities 
spacy_pharma = nlp(pharma)

# check this worked
#type(spacy_pharma)
spacy_pharma.ents[0:4]

(4909, Insys Therapeutics Inc., today, Fentanyl)

In [9]:
## B

# use list iteration to print the unique named entities with the tag: LAW.
law_tags = [token.text for token in spacy_pharma.ents if token.label_ == "LAW"]
law_tags

['RICO', 'the Controlled Substances Act', 'RICO']

C. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years. Print these named entities.

In [10]:
## C

# Pull out the named entities with 
# (1) the label DATE and 
# (2) that contain the word year or years. 
year_pattern = r"year|years"     # create a pattern to search for year or years
year_tokens = [token.text for token in spacy_pharma.ents if token.label_ == "DATE" 
             and re.search(year_pattern, token.text) is not None]
year_tokens  # print matches

['last year', 'three years', 'three years']

D. Parse the pharma string at the sentence level. Note that this involves more than just splitting on each `.`; for full credit, add at least one additional delimiter that marks the end of the sentence.

Then, using those sentences, pull and print the original sentences from the press releases where those year lengths are mentioned. Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

In [11]:
pharma

'4909    The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RIC

In [12]:
## D
# Parse the pharma string at the sentence level.

# create a pattern that looks for periods, except those:
# 1) preceded by a single capital letter, or
# 2) part of "U.S.", or
# 3) followed by a "," 
sent_pattern = r"(?<!\s[A-Z])(?<!U\.S)\.[^,]"    #https://www.regular-expressions.info/lookaround.html

# split the string based on this pattern
sent_split = re.split(sent_pattern, pharma) 
#sent_split

# Pull and print the original sentences from the press releases where those year lengths are mentioned. 
year_tokens_pattern = r"last year|three years"      # create pattern from the unique year tokens we found before
year_quotes = [sentence for sentence in sent_split if re.search(year_tokens_pattern, sentence) is not None]  
year_quotes   # print matches

# Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) 
# the CEO may be facing if convicted after this indictment 
# (if there are multiple lengths mentioned describe the maximum).

# Response: The CEO may face up to five years in prison and three years of propation for the Anti-Kickback Law 
# as well as up to a total of 20 years in prison and three years propoation for RICO and 
# the same for conspiracy to commit mail and wire fraud.

['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids',
 'he charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss',
 ' The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine']

## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [13]:
## A

# Subset the press releases to those labeled with one of three topics via topics_clean: 
# Civil Rights, Hate Crimes, and Project Safe Childhood. 
doj_subset = doj.loc[doj.topics_clean.isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]

# Check the shape
doj_subset.shape

(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint:** you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [14]:
## B
# Part 1: define a new function

def polarity_no_ents(one_string): 
    '''
    Function to return the polarity_scores/sentiment dictionary of a string after removing named entities
    
    Parameters:
        str: press release
        
    Returns: 
        pd.DataFrame: polarity_scores/sentiment dictionary 
    '''
    spacy_press = nlp(one_string)      # find named entities 
    press_ent = [ent.text for ent in spacy_press.ents]     #take the string for each entity
    result = re.sub("|".join([re.escape(ent.text) for ent in spacy_press.ents]),
                    " ", one_string)     # re.sub to replace any ents
    sent_obj = SentimentIntensityAnalyzer()     # name the analyzer
    return sent_obj.polarity_scores(result)     # return the polarity scoes

#https://predictivehacks.com/how-to-run-sentiment-analysis-in-python-using-vader/

In [15]:
# B Part 2: execute this function
sentiment_scores_results = doj_subset.contents.apply(polarity_no_ents)

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score (so most negative to least negative) and print the `id`, `contents`, and `neg` columns of the two most negative press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [16]:
## C

# Add the four sentiment scores to the doj_subset dataframe to create a dataframe: doj_subset_wscore.

# convert each dict to a string 
doj_subset['sentiment_scores'] = [str(dict) for dict in sentiment_scores_results]

# split the sentiment_scores column to four new columns 
doj_subset[['neg', 'neu', 'pos', 'compound']] = doj_subset.sentiment_scores.str.split(",", expand = True) 

# rewrite the columns with regex to keep only the numbers
pattern_int = r"\d*(-?[0-9]\.\d+)"
# https://stackoverflow.com/questions/15814592/how-do-i-include-negative-decimal-numbers-in-this-regular-expression
doj_subset['neg'] = doj_subset.neg.str.extract(pattern_int) 
doj_subset['neu'] = doj_subset.neu.str.extract(pattern_int) 
doj_subset['pos'] = doj_subset.pos.str.extract(pattern_int) 
doj_subset['compound'] = doj_subset.compound.str.extract(pattern_int) 

# check to see this worked
#doj_subset[['id', 'title', 'sentiment_scores', 'neg', 'neu','pos', 'compound']].head()

# name as new df
doj_subset_wscore = doj_subset.drop(['sentiment_scores'], axis = 1).copy()

# check this worked
#doj_subset_wscore.head()
#type(doj_subset_wscore.neg)

# Sort from highest neg to lowest neg score (so most negative to least negative) 
neg_sort_desc = doj_subset_wscore.sort_values(by = ['neg'], ascending = False).copy()

# Print the id, contents, and neg columns of the two most negative press releases.
neg_sort_desc[['id', 'contents', 'neg']].iloc[0:2].copy()

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.33
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.301


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [17]:
#D
# agg and find the mean compound score by topic

# first need to convert scored columns to numeric (float) values
doj_subset_wscore['neg'] = doj_subset_wscore['neg'].astype(float) 
doj_subset_wscore['neu'] = doj_subset_wscore['neu'].astype(float) 
doj_subset_wscore['pos'] = doj_subset_wscore['pos'].astype(float) 
doj_subset_wscore['compound'] = doj_subset_wscore['compound'].astype(float) 

# check this worked
doj_subset_wscore.info()

# then agg and find the mean compound score by topic
mean_compound = doj_subset_wscore.groupby("topics_clean").agg({"compound": "mean"}).copy()
mean_compound

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717 entries, 77 to 13081
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                707 non-null    object 
 1   title             717 non-null    object 
 2   contents          717 non-null    object 
 3   date              717 non-null    object 
 4   topics_clean      717 non-null    object 
 5   components_clean  717 non-null    object 
 6   neg               717 non-null    float64
 7   neu               717 non-null    float64
 8   pos               717 non-null    float64
 9   compound          717 non-null    float64
dtypes: float64(4), object(6)
memory usage: 61.6+ KB


,compound
topics_clean,
Civil Rights,-0.093398
Hate Crimes,-0.935567
Project Safe Childhood,-0.664434


**E. RESPONSE:** As we can see, hate crimes have the strongest negative mean since the value is closest to 1, although all three topics are closer to the the most negative value (-1). 

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in the code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string (so if press release is something like "The CEO was indicted" it might return "ceo indict" 
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`. Note: there will be a deduction if your code uses a non-list comprehension for loop that uses append.
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)

In [18]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]
stop_words = stopwords.words('english')
stop_words_new = stop_words + custom_doj_stopwords

In [19]:
## A Define a text processing function

def text_process(one_string):
    '''Takes in a single raw string and does the following preprocessing steps: 
    1) Converts the words to lowercase
    2) Removes stopwords, adding the custom stopwords in the code cell below to the default stopwords list' 
    3) Only retains alpha words (so removes digits and punctuation)
    4) Only retains words 4 characters or longer
    5) Uses the snowball stemmer from nltk to stem
    
    Parameters:
        str
    
    Returns:
        a joined preprocessed string'''
    
    one_string.lower() # Converts the words to lowercase
    wordpunct_tokenize(one_string) # tokenize
    no_stop_string = [word for word in wordpunct_tokenize(one_string) # Removes stopwords (custome list above)
                      if word not in stop_words_new]
    snowball = SnowballStemmer(language = 'english')  # Uses the snowball stemmer from nltk to stem
    preprocessed = [snowball.stem(token) for token in no_stop_string if 
                    token.isalpha() and   # Only retains alpha words (so removes digits and punctuation)
                    len(token) > 3]  # Only retains words 4 characters or longer
    clean_string = " ".join(preprocessed)
    return clean_string

In [20]:
## B
#  Use apply or list comprehension to execute that function and create a new column in the data called 
#  processed_text. 
#  Note: there will be a deduction if your code uses a non-list comprehension for loop that uses append.

doj_subset_wscore['processed_text'] = [text_process(string) for string in doj_subset_wscore.contents]

# test to see this worked
#doj_subset_wscore.head()

In [21]:
## C
# Print the id, contents, and processed_text columns for the following press releases:
example_subset = doj_subset_wscore[['id', 'contents', 'processed_text']].loc[doj_subset_wscore.id.isin(["16-718","16-217"])]
example_subset

,id,contents,processed_text
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",justic depart reach comprehens settlement agreement citi miami miami polic depart resolv justic depart offic involv shoot offic announc princip deputi assist attorney general vanita gupta head justic depart civil right divis attorney wifredo ferrer southern district florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem justic depart offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor under settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ this settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi assist attorney general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti par

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset two.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

In [22]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    metadata.columns = ["metadata_" + col for col in metadata.columns] #added in column renaming 
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [23]:
## A 
# Use the create_dtm function I provide and create a document-term matrix using the preprocessed press releases; 
# make sure metadata contains the following columns: id, compound sentiment column you added, 
   # and the topics_clean column
metadata = ["id", "compound", "topics_clean"]

doj_doc_matrix = create_dtm(list_of_strings = doj_subset_wscore.processed_text,
                        metadata = doj_subset_wscore[metadata])

doj_doc_matrix.head()

,index,metadata_id,metadata_compound,metadata_topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,77,17-1235,-0.9931,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,155,15-1522,-0.9325,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,157,16-213,-0.7579,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,162,16-381,-0.9037,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,168,14-464,-0.9864,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# define function to find top words of this subset
def top_words(df_subset):
    
    '''Takes a subset dataframe, sums the columns, 
    and and returns a series of the top 10 sums
    
    Parameters: subset pd.dataframe
    
    Returns: series'''
    
    top_terms = df_subset[[col for col in df_subset.columns
                               if "metadata" not in col and 
                               col != "index"]].sum(axis = 0)
    return top_terms.sort_values(ascending = False).head(10)


In [25]:
## B
# Print the top 10 words for press releases with compound sentiment in the top 5% 

# first, subset to greater than 95% compound scores
most_pos_subset = doj_doc_matrix[doj_doc_matrix.metadata_compound >= doj_doc_matrix.metadata_compound.quantile(0.95)].copy()
most_pos_subset.head()

# apply function 
top_words(most_pos_subset)

,index,metadata_id,metadata_compound,metadata_topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
35,683,11-645,0.9974,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,1857,17-271,0.9909,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,2394,16-525,0.9960,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,2402,16-1224,0.9948,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,2996,16-577,0.9933,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


depart       185
agreement    177
attorney     139
right        132
justic       130
enforc       121
civil        116
state        114
offic        110
disabl       108
dtype: int64

In [26]:
## C
# Print the top 10 words for press releases with compound sentiment in the bottom 5% 

# first, subset to less than 5% compound scores
most_neg_subset = doj_doc_matrix[doj_doc_matrix.metadata_compound <= doj_doc_matrix.metadata_compound.quantile(0.05)].copy()
most_neg_subset.head()

# apply function 
top_words(most_neg_subset)

,index,metadata_id,metadata_compound,metadata_topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
17,495,11-648,-0.9985,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,501,11-626,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,513,11-1275,-0.9980,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,572,13-312,-0.9983,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,1357,14-309,-0.9982,Hate Crimes,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


offic       245
attorney    213
assault     192
crime       179
victim      166
depart      156
right       152
divis       138
hate        134
defend      128
dtype: int64

In [27]:
## D

# Print the top 10 words for press releases in each of the three topics_clean
hate_crimes_subset = doj_doc_matrix.loc[doj_doc_matrix.metadata_topics_clean == "Hate Crimes"]
top_words(hate_crimes_subset)

civil_rights_subset = doj_doc_matrix.loc[doj_doc_matrix.metadata_topics_clean == "Civil Rights"]
top_words(civil_rights_subset)

safe_child_subset = doj_doc_matrix.loc[doj_doc_matrix.metadata_topics_clean == "Project Safe Childhood"]
top_words(safe_child_subset)

attorney    1086
right        656
divis        645
depart       628
victim       591
district     562
crime        557
civil        548
hate         524
defend       484
dtype: int64

depart       1122
right         949
attorney      942
offic         899
justic        894
civil         793
divis         772
district      670
hous          633
discrimin     616
dtype: int64

child          1022
attorney        878
exploit         701
district        589
sexual          572
safe            479
childhood       474
project         472
pornographi     452
depart          442
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled topics (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [ ]:
## A
# estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different 
# themes for each of the three manually-labeled topics (civil rights; hate crimes; project safe childhood). 

# Step 1: tokenize documents and store in list
processed_tokens = [wordpunct_tokenize(one_text) for one_text in doj_subset_wscore.processed_text]

# QUESTION: didn't we already do this? ANS: my function before joined them together again, so now we need to repeat

# Step 2: Use gensim create dictionary to get all unique words across the documents
text_raw_dict = corpora.Dictionary(processed_tokens)

# Step 3: filter out the very rare and very common words from the dict; feeding it n docs as lower and upper bounds
lower_bound = round(doj_subset_wscore.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.shape[0]*0.95)

#lower_bound
#upper_bound

# QUESTION: what is this doing? ANSWER: this is just finding the number of rows where the cutoff is for 5 percent etc. 

text_raw_dict.filter_extremes(no_below = lower_bound,
                              no_above = upper_bound)

# Step 4: apply dictionary to TOKENIZED texts
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in processed_tokens]

# Step 5: estimate a model by feeding it:
# 1) the corpus mapped to the dictionary
# 2) the dictionary itself 
# 3) the number of topics (often k if you're reading notation)
# 4) +  any other assorted other arguments

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                num_topics = 3, id2word=text_raw_dict, 
                                passes=50, alpha = 'auto',
                                per_word_topics = True, random_state = 91988)

In [ ]:
## B
# After estimating the topic model, print the top 15 words in each topic.
topics = ldamod.print_topics(num_words = 15)
for topic in topics:
    print(topic)

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). 
**Hint**:    
pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

In [ ]:
## A 

# Extract the document-level topic probabilities
topic_probs_bydoc = [ldamod.get_document_topics(item, minimum_probability = 0) for item in corpus_fromdict]

# Write an assert statement to make sure the length of the list is the same # of rows as our df
doj_subset_wscore.shape
assert len(topic_probs_bydoc) == 717

In [ ]:
## B 

# Add the topic probabilities to the doj_subset_wscores dataframe as columns and create a column (top_topic)
# that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

# check topic probs for one example
one_list_tup = topic_probs_bydoc[0]
one_list_tup

# step 1: create a long for dataframe by flattening the list
topic_probs_bydoc_long = pd.DataFrame([t for lst in topic_probs_bydoc for t in lst],
                                     columns = ['topic', 'probability'])

# step 2: add the id var - we're repeating each id in the original data k times for the number of topics 
topic_probs_bydoc_long['doc_id'] = list(np.concatenate([[one_id] * 3 
                                                      for one_id in doj_subset_wscore.id]).flat)

# step 3: pivot to wide format
topic_probs_bydoc_wide = pd.pivot_table(topic_probs_bydoc_long, index = ['doc_id'],
                                       columns = ['topic']).reset_index().reset_index(drop = True)

topic_probs_bydoc_wide.columns = ['doc_id'] + ["topic_" + str(i) for i in np.arange(0, 3)]

# step 4: merge with original data using doc id
topic_wmeta = pd.merge(topic_probs_bydoc_wide, 
                       doj_subset_wscore,
                       left_on = 'doc_id',
                       right_on = 'id')

# step 5: create indicator for listing's top topic
topic_wmeta['top_topic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                       "topic_" in col]].idxmax(axis = 1)

# check to see this worked 
#topic_wmeta.sample(5, random_state = 555)
topic_wmeta_small = topic_wmeta.drop(["contents", "date", "processed_text"], axis = 1).copy()
topic_wmeta_small.head()

In [ ]:
## C 
# your code here to summarize the topic proportions for each of the topics_clean 
#  For each of the manual labels in topics_clean (Hate Crime, Civil Rights, Project Safe Childhood), 
#print the breakdown of the % of documents with each top topic 
# (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, 
#that would be 50%; and so on).

# First try
topic_wmeta_small.groupby("topics_clean").agg({"topic_0": np.mean,
                                              "topic_1": np.mean, 
                                              "topic_2": np.mean})

# Second try
# Hint: pd.crosstab and normalize may be helpful
pd.crosstab(topic_wmeta_small.topics_clean, topic_wmeta_small.top_topic, normalize = "index")

In [ ]:
## D

# Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of 
# the manual topics map on more cleanly to an estimated topic than other manual topic(s)
topic_wmeta.columns
#topic_wmeta.head()
topic_wmeta_clean = topic_wmeta.drop(["doc_id", "date", "neg", "neu", "pos", "compound", "processed_text"], 
                                     axis = 1).copy()

# Example 1:
topic_wmeta_clean[topic_wmeta_clean.topics_clean == "Hate Crimes"].sample(1)

# Example 2: 
topic_wmeta_clean[topic_wmeta_clean.topics_clean == "Civil Rights"].sample(1)

# Example 3: 
topic_wmeta_clean[topic_wmeta_clean.topics_clean == "Project Safe Childhood"].sample(1)


**D. Response:** Although topic 1 corresponds well to "Project Safe Childhood", it is less precise to match the manual topics to topic_2 and topic 3 (see the top_topic values for topic Example 1 and Example 2 printed below). This is likely due to the overlap of the nature of hate crimes and civil rights crimes and that a civil rights division (components_clean) exists for the hate crime topic.  

# 3. Extend the analysis from unigrams to bigrams (9 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:
## A 
# Using the doj_subset_wscore data and the processed_text column create a column in the data 
# called processed_text_bigrams that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:
# Do this by writing a function create_bigram_onedoc that takes in a single processed_text string and 
# returns a string with its bigrams structured similarly to above example

In [ ]:
## A

# Step 1: define the function create_bigram_onedoc

def create_bigram_onedoc(one_string):
    '''Takes in a single processed_text string and returns a string with its bigrams structured 
    similarly so that that each consecutive pairs of word is combinedinto a bigram separated by an underscore
    '''
    split_list = one_string.split(" ")   # split on space
    bigrams = [str(first + "_" + second) for first, second in zip(split_list, split_list[1:])] # create bigram
    return " ".join(bigrams)   # rejoin into one list

In [ ]:
## A 

# Step 2: run the function and save as another column
doj_subset_wscore["processed_text_bigrams"] = [create_bigram_onedoc(string) for string 
                                               in doj_subset_wscore.processed_text]

# test this worked
#doj_subset_wscore.head()

In [ ]:
## B

# Print the id, processed_text, and processed_text_bigram columns for press release with id = 16-217
doj_subset_wscore[["id", "processed_text", "processed_text_bigrams"]].loc[doj_subset_wscore.id == "16-217"]

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the 
 (1) dimensions of the `dtm` matrix from question 2.2  and 
 (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2


In [ ]:
## C

# Use the create_dtm function and the processed_text_bigrams column to create a document-term matrix (dtm_bigram)
# with these bigrams. Keep the following three columns in the data: id, topics_clean, and compound
dtm_bigram = create_dtm(list_of_strings = doj_subset_wscore.processed_text_bigrams,
                       metadata = doj_subset_wscore[["id", "topics_clean", "compound"]])

In [ ]:
## D

# Print the dimensions of the dtm matrix from question 2.2 
doj_doc_matrix.shape

# Print the dimensions of the dtm_bigram matrix. 
dtm_bigram.shape

# Comment on why the bigram matrix has more dimensions than the unigram matrix
# Response: The bigram matrix has more columns because in creating bigrams, we vastly increased the number of 
# potential tokens.

In [ ]:
## E 

# Find and print the 10 most prevelant bigrams for each of the three topics_clean using the get_topwords 
# function from 2.2
hate_crimes_subset_b = dtm_bigram.loc[dtm_bigram.metadata_topics_clean == "Hate Crimes"]
top_words(hate_crimes_subset_b)

civil_rights_subset_b = dtm_bigram.loc[dtm_bigram.metadata_topics_clean == "Civil Rights"]
top_words(civil_rights_subset_b)

safe_child_subset_b = dtm_bigram.loc[dtm_bigram.metadata_topics_clean == "Project Safe Childhood"]
top_words(safe_child_subset_b)